In [1]:
# import the necessary packages
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ast import literal_eval
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import re
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import vgg16
from tensorflow.keras import models, Model
from numpy import save
from numpy import load
from deep_translator import GoogleTranslator
import langid
import nltk

2021-12-31 12:59:31.024980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-31 12:59:31.025316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/phd/rina_1921cs13/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Utility Functions
def get_image(imagepath):
	img = Image.open(imagepath)
	img = img.resize((224,224))
	img = img_to_array(img)
	if img.shape[2]==1:
		img = np.stack([img,img,img],axis=2)
		img = img.reshape(img.shape[0],img.shape[1],3)
	return img

def cleaned_text(x):
	x = str(x)
	val =  [re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() ]  # if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words
	res = ""
	for i in range(min(300,len(val))):
		res = res + val[i] + ' '
	res = res.split()
	res = ' '.join(res)
	return res

def clean_para(text):
	sent_text = nltk.sent_tokenize(text)
	cleaned_sentences = []
	for i in range(min(len(sent_text), 15)):
		cleaned_sentences.append(cleaned_text(sent_text[i]))
	cleaned_para = '. '.join(cleaned_sentences)
	return cleaned_para

In [4]:
# get data
def make_pairs():
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairTexts = []
    pairLabels = []
    ID = []
    print('[INFO] Loading and Processing Dataset...')
    source = []
    temp = pd.read_csv('/Source/Snopes.csv')
    source.append(temp)
    temp = pd.read_csv('/Source/Reuters.csv')
    source.append(temp)
    temp = pd.read_csv('/Source/ReCovery.csv')
    source.append(temp)
    temp = pd.read_csv('/Source/TICNN.csv')
    source.append(temp)
    source = pd.concat(source, ignore_index=True, sort=False)
    target = []
    temp = pd.read_csv('/Target/Snopes.csv')
    target.append(temp)
    temp = pd.read_csv('/Target/Reuters.csv')
    target.append(temp)
    temp = pd.read_csv('/Target/ReCovery.csv')
    target.append(temp)
    temp = pd.read_csv('/Target/TICNN.csv')
    target.append(temp)
    target = pd.concat(target, ignore_index=True, sort=False)
    print('Source columns', source.columns)
    print('Target columns', target.columns)
    source = source.iloc[:,:].values
    target = target.iloc[:,:].values
    return source, target

In [5]:
source, target = make_pairs()

[INFO] Loading and Processing Dataset...
Source columns Index(['Unnamed: 0', 'ID', 'numSources', 'Source_url1', 'Source_text1',
       'Image_url1', 'Source_reliability1', 'Source_url2', 'Source_text2',
       'Image_url2', 'Source_reliability2', 'Source_url3', 'Source_text3',
       'Image_url3', 'Source_reliability3', 'Source_url4', 'Source_text4',
       'Image_url4', 'Source_reliability4'],
      dtype='object')
Target columns Index(['Unnamed: 0', 'ID', 'Target_url', 'Target_text', 'Image_url', 'label'], dtype='object')


In [6]:
print('Source type', type(source))
print('Source shape', source.shape)
print('Target type', type(target))
print('Target Shape', target.shape)

Source type <class 'numpy.ndarray'>
Source shape (6272, 19)
Target type <class 'numpy.ndarray'>
Target Shape (6272, 6)


In [7]:
# get data
def get_data(source, target):
    pairImages = []
    pairTexts = []
    pairLabels = []
    ID = []
    identity = []
    ctr = 0
    for i in range(len(source)):
        if i%100==0:
            print('Loading Data...',i)
        target_img = ''
        try:
            target_img = get_image('/TargetImages/' + source[i][1] + '.jpg')
        except:
            continue
        target_txt = cleaned_text(target[i][3])
        dataset = ''
        if source[i][1][0:3]=='Snp':
            dataset = 'Snopes'
        elif source[i][1][0:3]=='Rtr':
            dataset = 'Reuters'
        elif source[i][1][0:3]=='Rcv':
            dataset = 'ReCovery'
        else:
            dataset = 'TICNN'
        for j in range(3,3+4*source[i][2],4):
            try:
                source[i][j+2] = literal_eval(source[i][j+2])
            except:
                pass
            src_txt = clean_para(source[i][j+1])
            # src_txt = source[i][j+1]
            try:
                src_img = get_image('/SourceImages/' + dataset + '/' + source[i][j+2]['image_name'])
                pairImages.append([target_img, src_img])
                pairTexts.append([target_txt, src_txt])	
                ID.append([source[i][j+2]['image_name']])
                identity.append(ctr)			
                if target[i][5]=='FAKE':
                    pairLabels.append([0])
                else:
                    pairLabels.append([1])
            except:
                continue
        ctr +=1 
    # return a 2-tuple of our pairs and labels
    return (np.array(pairImages), np.array(pairTexts), np.array(pairLabels), np.array(ID), np.expand_dims(np.array(identity), 1))

In [8]:
img_pair, txt_pair, label, ID, identity = get_data(source, target)

Loading Data... 0
Loading Data... 100
Loading Data... 200
Loading Data... 300
Loading Data... 400
Loading Data... 500
Loading Data... 600
Loading Data... 700
Loading Data... 800
Loading Data... 900
Loading Data... 1000
Loading Data... 1100
Loading Data... 1200
Loading Data... 1300
Loading Data... 1400
Loading Data... 1500
Loading Data... 1600
Loading Data... 1700
Loading Data... 1800
Loading Data... 1900
Loading Data... 2000
Loading Data... 2100
Loading Data... 2200
Loading Data... 2300
Loading Data... 2400
Loading Data... 2500
Loading Data... 2600
Loading Data... 2700
Loading Data... 2800
Loading Data... 2900
Loading Data... 3000
Loading Data... 3100
Loading Data... 3200
Loading Data... 3300
Loading Data... 3400
Loading Data... 3500
Loading Data... 3600
Loading Data... 3700
Loading Data... 3800
Loading Data... 3900
Loading Data... 4000
Loading Data... 4100
Loading Data... 4200
Loading Data... 4300
Loading Data... 4400
Loading Data... 4500
Loading Data... 4600
Loading Data... 4700
Load

In [15]:
# Data analysis
print(img_pair.shape)
print(txt_pair.shape)
print(label.shape) # 0 - Fake, 1 - Real
print(ID.shape)
print(identity.shape)
# Shape: num_samples, 2, features -> 2 is for the source and the image

(11766, 2, 224, 224, 3)
(11766, 2)
(11766, 1)
(11766, 1)
(11766, 1)


In [16]:
print('ID\n', ID[:5,:])
print('Identity\n', identity[:5,:])

ID
 [['Snp1_3_1.jpg']
 ['Snp3_1_19.jpg']
 ['Snp3_2_4.jpg']
 ['Snp3_3_9.jpg']
 ['Snp3_4_9.jpg']]
Identity
 [[0]
 [1]
 [1]
 [1]
 [1]]


In [17]:
# Saving the identity
np.save('data/identity', identity) 

In [20]:
txt_pair[3]

array(['a photograph shows four democratic congresswomen posing with a portrait of osama bin laden and an isis flag',
       'marion mcgarry a member of the board of representatives for stamford s th district smiles as she chats with guests at the first congregational church s thanksgiving day dinner in stamford conn on thursday nov. marion mcgarry d stamford democratic chairman josh fedeli at left watches as board of representatives for the city of stamford vote on the censure of rep marion mcgarry during a special meeting in the legislative chambers of the government center in stamford conn on tuesday march. a facebook post from the account of marion mcgarry a democrat on the stamford board of representatives. a facebook post from the account of marion mcgarry a democrat on the stamford board of representatives. a facebook post from the account of marion mcgarry a democrat on the stamford board of representatives. a facebook post from the account of marion mcgarry a democrat on the s

In [21]:
label

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [28]:
# Text analysis
import copy
print(txt_pair[0]) # source sentence is not in English
#TODO: Force translate all text sentences to English
translated_txt_pairs = []
for i in range(len(txt_pair)):
    lang_target = langid.classify(txt_pair[i][1])[0]
    if lang_target != 'en':
        print('In here', i)
        to_translate = txt_pair[i][1]
        if len(to_translate) >= 5000:
            to_translate = to_translate[:4999]
        print(len(to_translate))
        trans_source = GoogleTranslator(source='auto', target='english').translate(to_translate)
    else:
         trans_source = txt_pair[i][1]
    translated_txt_pairs.append([txt_pair[i][0], trans_source])
trans_txt_pair = np.array(translated_txt_pairs)

['fifteen homeless people in chicago were found dead on the street because of record low temperatures that hit the city in late january'
 'quindici senzatetto a chicago sarebbero stati trovati morti per strada a causa delle temperature record che hanno colpito la citt alla fine di gennaio. quando la devastante tempesta invernale ha attraversato chicago a fine gennaio gli utenti dei social media forse preoccupati sono caduti in errore con un meme che ha gonfiato il bilancio della tempesta sulla citt. il post che ha avuto origine su twitter il gennaio recitava senzatetto di chicago sono morti congelati. prenditi un momento e pensa a quanto sei fortunato in questo momento. dimentica il muro di trump le mosse della pelosi il jordan che vuoi il nuovo borsellino che vuoi. per un momento pensa a quanto sei fortunato e benedetto. quanto siamo benedetti. mentre il tweet stato visualizzato pi di volte con pi di likes la foto di accompagnamento raffigurante un senzatetto che dorme su una strada i

In [29]:
lang_target = langid.classify(txt_pair[0][0])[0]
print(lang_target != 'en')
print(lang_target)

False
en


In [30]:
print(img_pair.shape)
print(trans_txt_pair.shape)
print(label.shape) # 0 - Fake, 1 - Real

(11766, 2, 224, 224, 3)
(11766, 2)
(11766, 1)


In [31]:
np.save('data/image_array', img_pair)
np.save('data/text_array_new', trans_txt_pair)
np.save('data/labels', label)
np.save('data/ids', ID)

In [ ]:
img_array_load = np.load('data/image_array.npy')
txt_array_load = np.load('data/text_array_new.npy')
labels_load = np.load('data/labels.npy')
ids_load = np.load('data/ids.npy')

In [34]:
txt_array_load = np.load('data/text_array_new.npy')
print(txt_array_load.shape)

(11766, 2)


In [39]:
txt_array_load[5]

array(['a photograph shows medalist jim thorpe wearing mismatched shoes at the olympics',
       'james francis thorpe sac and fox sauk wa tho huk translated as bright path may or march was an american athlete and olympic gold medalist. a member of the sac and fox nation thorpe became the first native american to win a gold medal for the united states. considered one of the most versatile athletes of modern sports he won two olympic gold medals in the summer olympics one in classic pentathlon and the other in decathlon and played american football collegiate and professional professional baseball and basketball. he lost his olympic titles after it was found he had been paid for playing two seasons of semi professional baseball before competing in the olympics thus violating the amateurism rules that were then in place. in years after his death the international olympic committee ioc restored his olympic medals with replicas after ruling that the decision to strip him of his medals fell

In [ ]:
print(img_array_load.shape)
print(txt_array_load.shape)
print(labels_load.shape) # 0 - Fake, 1 - Real
print(ids_load.shape)